[View in Colaboratory](https://colab.research.google.com/github/awenare/xyhacker/blob/master/Amazon_NLTK_Baseline_Model_V4.ipynb)

In [0]:
import numpy as np
import pandas as pd

In [2]:
!wget -nc https://www.dropbox.com/s/gp1vslen8vnzvvl/genders_data.csv?raw=1


Redirecting output to ‘wget-log’.


In [3]:
df = pd.read_csv('genders_data.csv?raw=1', index_col = 0)
df.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,names,review,summary,gender
0,gsxrgirl,"No sugar, no GMO garbage, no fillers that come...",Best vanilla I've ever had,unknown
1,FIFA Lvr,"This is my absolute, undisputed favorite tea r...",Terrific Tea!,unknown
2,Alicia b,I ordered spongbob slippers and I got John Cen...,grrrrrrr,female
3,"Danny K. Tilley ""Dan Tilley""",The cart is fine and works for the purpose for...,Storage on Wheels Cart,male
4,CHelmic,This product by Archer Farms is the best drink...,The best drink mix,unknown


In [0]:
df.drop(df[df.gender == 'unknown'].index, inplace=True) #In simple terms, index() method finds the given element in a list and returns its position.
df.reset_index(inplace = True)
df.head()
df.dropna(inplace = True)

In [5]:
new_df = df.drop(columns  = ['index'])
new_df.head()

,names,review,summary,gender
0,Alicia b,I ordered spongbob slippers and I got John Cen...,grrrrrrr,female
1,"Danny K. Tilley ""Dan Tilley""",The cart is fine and works for the purpose for...,Storage on Wheels Cart,male
2,"Trevor L ""God is on the side of the army with...",My wife picked some of this up on sale. I usu...,Tastes a Bit like Cough Syrup,male
3,"Mary S ""one800mary""",I had a martini at a local distillery that use...,Interesting bitters,female
4,Jon Stoffel,And they're pretty nice! One set looks like ic...,They're dice!,male


In [6]:
df.count()

index      618852
names      618852
review     618852
summary    618852
gender     618852
dtype: int64

In [0]:
chunk = new_df[:10000]

In [0]:
review_chunk = chunk['review']

# NLTK 

In [9]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
by_hand_pos_keys = ['$', "''","(", ')', ',', '--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP','NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG','VBN','VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``', '#']
len(by_hand_pos_keys)

48

In [0]:
from nltk.tokenize import sent_tokenize

In [0]:
zee_data=[]
for i in review_chunk:
  try:
    sent_tokenize_list = sent_tokenize(i)
    array = np.zeros(48)
    for item in sent_tokenize_list:
      try:
        tokenized = nltk.word_tokenize(item)
        tagged = nltk.pos_tag(tokenized)
        tags = [b[1] for (a, b) in nltk.bigrams(tagged)]
        for tag in tags:
          index = by_hand_pos_keys.index(tag)
          array[index] +=1
      except:
        continue
    zee_data.append(array)
  except:
    print(i)
    continue

In [0]:
genders = new_df['gender']
gender_chunk = genders[:10000]
zee_data_Y = []
for gender in gender_chunk:
  if gender == 'male':
    zee_data_Y.append(1)
  else:
    zee_data_Y.append(0)

In [14]:
print(len(zee_data_Y))
print(len(zee_data))

10000
10000


Multinomial NB = 0.5786666666666667 / 0.5843333333333334

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(zee_data, zee_data_Y, test_size=0.3)

In [16]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.5833333333333334

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
clf_pf = GaussianNB()
clf_pf.partial_fit(X_train, y_train, np.unique(Y))
GaussianNB(priors=None)
print(clf_pf.predict([[-0.8, -1]]))

### Gradient Descent

(continuation of Susanna's work)

[Link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

In [0]:
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier

In [61]:

X = X_train
Y = y_train
clf = linear_model.SGDClassifier()
clf.fit(X, Y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

clf.score(X, Y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5605714285714286

## Kelvin's Latest Dataset Test

13/9/18

In [20]:
!wget -nc https://www.dropbox.com/s/ezl4oj16vkeu21l/supreme_genders_data.pkl?dl=0
data = pd.read_pickle('supreme_genders_data.pkl?dl=0')
df = pd.DataFrame(data)


Redirecting output to ‘wget-log.1’.


In [0]:
data = pd.read_pickle('supreme_genders_data.pkl?dl=0')
df = pd.DataFrame(data)

In [23]:
df = pd.DataFrame(data)
df.shape

(1289462, 4)

In [24]:
df.head()

,names,review,summary,gender
0,gsxrgirl,"No sugar, no GMO garbage, no fillers that come...",Best vanilla I've ever had,female
1,FIFA Lvr,"This is my absolute, undisputed favorite tea r...",Terrific Tea!,unknown
2,Alicia b,I ordered spongbob slippers and I got John Cen...,grrrrrrr,female
3,"Danny K. Tilley ""Dan Tilley""",The cart is fine and works for the purpose for...,Storage on Wheels Cart,male
4,CHelmic,This product by Archer Farms is the best drink...,The best drink mix,unknown
